In [1]:
import numpy as np
np.random.seed(13)
import pandas as pd
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras import optimizers
from __future__ import division
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

def findMaxAcc(valacc,valloss):
	prevloss=valloss[0]
	for i in range(1,len(valloss)):
		if(valloss[i]>prevloss):
			return valacc[i-1]
	#print('Problems with find MaxAccuracy')
	return valacc[len(valacc)-1]

ticker='AAPL'
y=[]
price=pd.read_csv('/home/andrea/Desktop/NLFF/indexesLabeledNew/indexes'+ticker+'.csv')
#print(price)
trendwindowtime=[1,4,20]
price=price.iloc[2500:]
#xtemp=price[['open','close','volume','macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6']]
xtemp=price[['open','close']]
xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    
    for i in range(0,len(price)-t):
        y.append(np.sign(price.iloc[i+t]['close']-price.iloc[i]['open']))
        x.append(xtemp[i])
    y=np.array(y)
    x=np.array(x)
    y=(y+1)/2
    
    #Split betwwen train-validation and test
    
    train=0.8
    nt=math.ceil(len(x)*train)
    x_tv=[]
    y_tv=[]
    x_test=[]
    y_test=[]
    x_tv=x[:nt]
    y_tv=y[:nt]
    x_test=x[nt:]
    y_test=y[nt:]
    print(len(x_test))
    print(len(y_test))
    
    #Fairly sampling the tv
    posindex=np.where( y_tv == 1 )
    negindex=np.where( y_tv == 0 )
    
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))
    
    #for i in range(1,nindex):
    y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
    x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))
    
    #Fairly sampling the test 50% 50%
    posindex=np.where( y_test == 1 )
    negindex=np.where( y_test == 0 )
    
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))
    
    #for i in range(1,nindex):
    

    y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
    x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))
    
    nfold=10
    kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


    #model selection
    learningrate=[0.1,0.01,0.001,0.0001]
    drop=[0]
    neurons=[5,16,64]
    bestacc=0
    bestl=0
    bestdrop=0
    bestneuron=0
    
    cvaccuracy=[]
    print('ModelSelection')
    for l in learningrate:
        for d in drop:
            for n in neurons:
                #print(l)
                print()
                opt=keras.optimizers.SGD(lr=l, momentum=0.0, decay=0.0, nesterov=False)
                model = Sequential()
                model.add(Dense(n, input_shape=(2,), activation='sigmoid'))
                model.add(Dropout(d))
                model.add(Dense(n, activation='sigmoid'))
                model.add(Dropout(d))
                model.add(Dense(1, activation='sigmoid'))
                model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
                cvaccuracy=[]
                #crossvalidation
                for train_index, test_index in kf.split(x_tvnew):
                    x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                    y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                    history=model.fit(x_train, y_train, epochs=20, verbose=0,batch_size=10,validation_data=(x_val, y_val))

                    maxAcc=findMaxAcc(history.history['val_acc'],history.history['val_loss'])
                    cvaccuracy.append(maxAcc)
                mcvaccuracy=sum(cvaccuracy)/len(cvaccuracy)

                if(mcvaccuracy>bestacc):
                    print(mcvaccuracy)
                    bestacc=mcvaccuracy
                    bestl=l
                    bestdrop=d
                    bestneuron=n
                   
    print('Best Accuracy on validation:',bestacc)
    print('BestLearning:',bestl)
    print('BestDrop:',bestdrop)
    print('BestNeuron:',bestneuron)
    tot=[]
    print('LOOK HOW IT WORKS WITH TEST SET....')
    opt=keras.optimizers.SGD(lr=bestl, momentum=0.0, decay=0.0, nesterov=False)
    model = Sequential()
    model.add(Dense(bestneuron, input_shape=(2,), activation='sigmoid'))
    model.add(Dropout(bestdrop))
    model.add(Dense(bestneuron, activation='sigmoid'))
    model.add(Dropout(bestdrop))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
    model.fit(x_tvnew, y_tvnew, epochs=20, verbose=0,batch_size=10)
    evaluation=model.evaluate(x=x_testnew, y=y_testnew, batch_size=10, verbose=0)   
    acc=evaluation[1]
    print('Accuracy on test set')
    print(acc)

/home/andrea/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Working on window: 1
554
554
ModelSelection

0.5188545151863875











0.5347036423731633
Best Accuracy on validation: 0.5347036423731633
BestLearning: 0.0001
BestDrop: 0
BestNeuron: 64
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5000000000274929
Working on window: 4
554
554
ModelSelection

0.5140879397648364








0.5171080402893262



0.5191080404177832
Best Accuracy on validation: 0.5191080404177832
BestLearning: 0.0001
BestDrop: 0
BestNeuron: 64
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5000000004481552
Working on window: 20
551
551
ModelSelection

0.5011655265263553

0.5113451673962879




0.53629526709912


0.5634908759446524




0.6298189334175144
Best Accuracy on validation: 0.6298189334175144
BestLearning: 0.0001
BestDrop: 0
BestNeuron: 64
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.49999999953973245
